In [ ]:
# NOTE:  Probably will need to alter this to do sql queries that include joins, as 
# the column name issue will become trickier

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import seaborn as sns
import pymssql as sql

In [2]:
# Make Sure to Connect to New ARIS VPN first
# Connects to the ARIS database using my username and password

db = sql.connect(server='192.168.88.1\sql2014', user='dustin_sql', password='cchrc@123', 
                      database='akrebate')

In [3]:
# Write the SQL Query here just like you would in Microsoft SQL Server

test_query = """
SELECT DISTINCT * 
FROM eLib.SpaceHeatingEquipment heat
WHERE heat.LibraryVersion LIKE '2015-03-30'
"""

In [4]:
# This function takes an SQL query and executes it, returning the data as a list of tuples
def make_query(query_string):
    cursor = db.cursor()
    cursor.execute(query_string)
    query_result = cursor.fetchall()
    
    return query_result

In [5]:
# This function takes a given SQL database table and returns the column names as a list
def get_columns(table_name):
    
    cursor = db.cursor()
    table_name = '\''+table_name+'\''
    cursor.execute("""
    SELECT name 
    FROM sys.columns
    WHERE object_id = OBJECT_ID({})
    """.format(table_name))
                  
    column_headers = cursor.fetchall()
    
    clean_columns=[]
    for col in column_headers:
        clean = col[0].replace('(','')
        cleaner = clean.replace(',)', '')
        clean_columns.append(cleaner)
                   
    return clean_columns

In [7]:
# This function takes the output from the SQL query and converts it to a pandas dataframe

def dataframe_wrap(list_of_tuples, clean_columns):
    framelist=[]
    for i, col_name in enumerate(clean_columns):
        framelist.append((col_name,[e[i] for e in list_of_tuples]))
    return pd.DataFrame.from_items(framelist)

In [11]:
columns = get_columns('eLib.SpaceHeatingEquipment')
space_heat_data = make_query(test_query)
query_result_df = dataframe_wrap(space_heat_data, columns)
query_result_df.head()

LibraryVersion  ID                              Name  \
0     2015-03-30   1        Electric Resistance Direct   
1     2015-03-30   2  Standard Ground Src HP, Hydronic   
2     2015-03-30   3           Non-vented Space heater   
3     2015-03-30   4              Conventional furnace   
4     2015-03-30   5               Conventional Boiler   

                                         Description Active  Order  \
0  Electric Baseboard, Radiant Panels, or Space H...   True      1   
1  Ground Source Heat Pump with Average Efficienc...  False      2   
2                             No duct work, no vents   True    180   
3  Pre-1980, natural draft, continuous pilot, no ...   True    165   
4  Pre 1980, natural draft, continuous pilot, no ...   True    140   

   EquipTypeID  DistribType  HeatPump  AFUE  SteadyAdjust  AuxPowerPct  
0            1            3         1   100             0          0.0  
1            1            2         2   425             0          0.0  
2            2            3         1    72             2          3.0  
3            2            1         1    75             5          4.4  
4            2            2         1    72             8          0.5